In [4]:
!pip install alpaca-py
#!pip install nltk
#!pip install vaderSentiment
#!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for alpaca-py from https://files.pythonhosted.org/packages/f5/d7/bc9e4fe8e4d2b4d879960872925c59e4fcb8dc244445341a800bfc3fd0d6/alpaca_py-0.13.3-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic<3.0.0,>=2.0.3 from https://files.pythonhosted.org/packages/0a/2b/64066de1c4cf3d4ed623beeb3bbf3f8d0cc26661f1e7d180ec5eb66b75a5/pydantic-2.5.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/65.2 kB ? eta -:--:--
     ------------------ --------------------- 30.7/65.2 kB ? eta -:--:--
     ---------------------------------------- 65.2/65.2 kB 1.2 MB/s eta 0:00:00
  Obtaining dependency information for sseclient-py<2.0.0,>=1.7.2 from https://files.pythonhosted.org/packages/49/58/97655efdfeb5b4eeab85b1fc5d3fa1023661246c2ab2a26ea8e47402d4f2/sseclient_py-1.8.0-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/1

In [5]:
import time

In [6]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass

In [7]:
from alpaca.trading.requests import MarketOrderRequest, LimitOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

In [8]:

# Your API credentials
api_key = 'PKEECQOFTN00XC2N9EQ6'
api_secret = 'h1xslxUKlbpJt4Tu4xWRHWiePEOl2Ne90Bq6w6ha'


In [9]:
# Initialize trading client for paper trading
trading_client = TradingClient(api_key, api_secret, paper=True)

# Define target profit and stop loss percentages
target_profit = 0.05  # 15%
stop_loss = -0.03    # -3%

In [10]:
def place_market_order(symbol, quantity, side):
    """
    Places a market order for the specified symbol and quantity.
    """
    order_data = MarketOrderRequest(symbol=symbol, qty=quantity, side=side, time_in_force=TimeInForce.GTC)
    return trading_client.submit_order(order_data=order_data)


In [11]:
def check_and_close_position(symbol, unrealized_plpc):
    """
    Closes the position for the specified symbol if target profit or stop loss is hit.
    """
    if unrealized_plpc >= target_profit or unrealized_plpc <= stop_loss:
        trading_client.close_position(symbol)


In [12]:
def monitor_positions():
    """
    Monitors all open positions and closes them based on profit or stop loss criteria.
    Also throttles API requests to avoid hitting the limit.
    """
    while True:
        positions = trading_client.get_all_positions()
        for position in positions:
            unrealized_plpc = float(position.unrealized_plpc)
            check_and_close_position(position.symbol, unrealized_plpc)
        time.sleep(30)  # Adjust as needed to manage API call frequency



In [ ]:
# Example of placing a market order
place_market_order('BTC/USD', 0.023, OrderSide.BUY)

# Start monitoring positions
monitor_positions()

In [ ]:
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass, AssetStatus

search_params = GetAssetsRequest(
    asset_class=AssetClass.CRYPTO,
    status=AssetStatus.ACTIVE
)
assets = trading_client.get_all_assets(search_params)


for asset in assets:
    print(f"Symbol: {asset.symbol}, Name: {asset.name}, Exchange: {asset.exchange}")


In [ ]:
#=======